<a href="https://colab.research.google.com/github/vanoha/Machine-Learning/blob/main/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [3]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [4]:
import numpy as np
import pandas as pd


class DecisionTree:
    class Node:
        left = None
        right = None
        best_t = None
        feature = None
        target = None

    def _entropy_(self, part):
        _, counts = np.unique(part, return_counts = True)

        probabilities = counts / counts.sum()
        entropy = sum(probabilities * -np.log2(probabilities))

        return entropy

    def _IG_(self, X, feature, t):
        part1 = X[X[feature] < t]
        part2 = X[X[feature] >= t]
        if len(part1) == 0 or len(part2) == 0:
            return float('-inf'), [], []
        size = len(X[self.target])
        ig = self._entropy_(X[self.target]) - (
                len(part1) / size * self._entropy_(part1[self.target]) + len(part2) / size * self._entropy_(
            part2[self.
                target]))
        return ig, part1, part2

    def __init__(self, max_depth = 5, max_leaf_nodes = 5, min_weight_fraction_leaf = 5):
        self.max_depth = max_depth
        self.max_leaf_nodes = max_leaf_nodes
        self.min_weight_fraction_leaf = min_weight_fraction_leaf

    def fit(self, X: pd.DataFrame, y: str):
        self.target = y
        self.root = self.get_node(X)

    def get_node(self, X: pd.DataFrame, current_depth = 0):

        if len(np.unique(X[self.target])) == 1 or current_depth == self.max_depth or len(
                X) <= self.min_weight_fraction_leaf:
            node = self.Node()
            node.target = X[self.target].value_counts().index[0]
            return node

        features = list(X.columns)
        features.remove(self.target)
        best_feature = features[0]
        best_IG = float('-inf')
        best_t = 0
        best_p1 = pd.DataFrame()
        best_p2 = pd.DataFrame()
        node = self.Node()
        
        for feature in features:
            grid = np.linspace(X[feature].min(), X[feature].max(), 10)
            for t in grid:
                IG, p1, p2 = self._IG_(X, feature, t)
                if IG > best_IG:
                    best_p1 = p1
                    best_p2 = p2
                    best_t, best_IG, best_feature = t, IG, feature

        node.left = self.get_node(best_p1, current_depth + 1)
        node.right = self.get_node(best_p2, current_depth + 1)
        node.best_t = best_t
        node.feature = best_feature
        return node

    def predict(self, X: pd.DataFrame):
        y = []
        for i in range(len(X)):
            elem = X.iloc[i]
            y.append(self.DFS(self.root, elem))
        return y

    def DFS(self, node: Node, x):
        if node.left is None and node.right is None:
            return node.target
        if x[node.feature] < node.best_t:
            if node.left is None:
                return node.target
            return self.DFS(node.left, x)
        else:
            if node.right is None:
                return node.target
            return self.DFS(node.right, x)


In [5]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()

df = pd.DataFrame(data = np.c_[iris['data'], iris['target']],
                  columns = iris['feature_names'] + ['target'])

df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df.drop(['target'], axis = 1, inplace = True)
df = df.sample(frac = 1)

df_train = df.head(100)
df_test = df.tail(50)
dt = DecisionTree()
dt.fit(df_train, 'species')
predict = dt.predict(df_test.drop(['species'], axis = 1))

k = 0
for i, elem in enumerate(list(df_test['species'])):
    if elem == predict[i]:
        k += 1

dtc = DecisionTreeClassifier()
dtc.fit(df_train.drop(['species'], axis = 1).to_numpy(), df_train['species'].to_numpy())
prd = dtc.predict(df_test.drop(['species'], axis = 1).to_numpy())

print("схожесть с готовым: ", accuracy_score(predict, prd))
print("правильность ответов: ", k / len(df_test))


схожесть с готовым:  1.0
правильность ответов:  0.96


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализуйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [17]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
pd.options.mode.chained_assignment = None  # default='warn'

random.seed(42)

df = pd.read_csv("churn.csv", index_col='RowNumber')
df.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)
df['Gender'] = pd.get_dummies(df['Gender'], drop_first=True)
Geography_dumies = pd.get_dummies(df['Geography'], drop_first=True)
df = df.drop(columns=['Geography'])
df = pd.concat([df, Geography_dumies], axis=1)

X_train, X_test = train_test_split(df, random_state = 42, test_size = 0.3)
y_test = X_test['Exited']
X_test.drop(['Exited'], axis=1, inplace=True)

N = 10
m = 6000
trees = [(DecisionTreeClassifier(max_depth=random.randint(3, 10)), X_train.sample(n=m)) for i in range(N)]
y_pred = []

for tree, data in trees:
    y_train = data['Exited']
    X_train = data.drop(['Exited'], axis = 1)
    X_train = X_train.sample(n = int((len(X_train.columns)) / 2), axis = 1)
    tree.fit(X_train, y_train)
    X_test_current = pd.DataFrame(X_test, columns = list(X_train.columns))
    y_pred.append(tree.predict(X_test_current))

res = []
for i in range(len(y_pred[0])):
    l = []
    for j in range(len(y_pred)):
        l.append(y_pred[j][i])
    res.append(np.bincount(l).argmax())

print(accuracy_score(y_test, res))


0.8076666666666666
